In [5]:
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array
import tinysegmenter
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# split a loaded document into sentences
def to_pairs(doc):
	lines = doc.strip().split('\n')
	pairs = [line.split('\t') for line in  lines]
	return pairs
 
# clean a list of lines
def clean_pairs(lines):
	cleaned = list()
	# prepare regex for char filtering
	re_print = re.compile('[^%s]' % re.escape(string.printable))
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for pair in lines:
		clean_pair = list()
		for i in range(2):
			if i == 0:
				line = pair[0]
				# normalize unicode characters
				line = normalize('NFD', line).encode('ascii', 'ignore')
				line = line.decode('UTF-8')
				# tokenize on white space
				line = line.split()           
				# convert to lowercase
				line = [word.lower() for word in line]
				# remove punctuation from each token
				line = [word.translate(table) for word in line]
				# remove non-printable chars form each token
				line = [re_print.sub('', w) for w in line]
				# remove tokens with numbers in them
				line = [word for word in line if word.isalpha()]
				# store as string
				clean_pair.append(' '.join(line))
			else:
				line = pair[1]
				l = list(line)[:-1]
				line = [word.lower() for word in l]
				line1 = "".join(line)
				tokenized_statement = tinysegmenter.tokenize(line1)                
				clean_pair.append(' '.join(tokenized_statement))
		cleaned.append(clean_pair)
	return array(cleaned)
 
# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)
 
# load dataset
filename = 'jpn.txt'
doc = load_doc(filename)
# split into english-german pairs
pairs = to_pairs(doc)
# print(pairs)
# clean sentences
clean_pairs = clean_pairs(pairs)
# print(clean_pairs)
# save clean pairs to file
save_clean_data(clean_pairs, 'english-japanese_simple2.pkl')
# spot check
for i in range(50):
	print('[%s] => [%s]' % (clean_pairs[i,0], clean_pairs[i,1]))
#     print(clean_pairs[i])

Saved: english-japanese_simple2.pkl
[go] => [行け]
[go] => [行き なさい]
[hi] => [こん にち は]
[hi] => [もしもし]
[hi] => [やっほー]
[hi] => [こん にち は]
[run] => [走れ]
[run] => [走っ て]
[who] => [誰]
[wow] => [すごい]
[wow] => [ワォ]
[wow] => [わぉ]
[wow] => [おー]
[fire] => [火事 だ]
[fire] => [火事]
[fire] => [撃て]
[help] => [助け て]
[help] => [助け て くれ]
[jump] => [飛び越えろ]
[jump] => [跳べ]
[jump] => [飛び降りろ]
[jump] => [飛び跳ね て]
[jump] => [ジャンプ し て]
[jump] => [跳べ]
[jump] => [飛び跳ね て]
[jump] => [ジャンプ し て]
[stop] => [やめろ]
[stop] => [止まれ]
[wait] => [待っ て]
[go on] => [続け て]
[go on] => [進ん で]
[go on] => [進め]
[go on] => [続けろ]
[hello] => [こん にち は]
[hello] => [もしもし]
[hello] => [こん にち は]
[hurry] => [急げ]
[i see] => [なる ほど]
[i see] => [なる ほどね]
[i see] => [わかっ た]
[i see] => [わかり まし た]
[i see] => [そう です か]
[i see] => [そう なん だ]
[i see] => [そっか]
[i try] => [頑張っ て みる]
[i try] => [やっ て みる]
[i try] => [試し て みる]
[i try] => [やっ て みよう]
[i try] => [トライ し て みる]
[i won] => [俺 の 勝ちー]


In [6]:
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# load dataset
raw_dataset = load_clean_sentences('english-japanese_simple2.pkl')
# print(raw_dataset)

# reduce dataset size
n_sentences = 10000
dataset = raw_dataset[:n_sentences, :]
# print(dataset)
# random shuffle
shuffle(dataset)
# print(dataset)
# split into train/test
train, test = dataset[:9000], dataset[9000:]
# save
save_clean_data(dataset, 'english-japanese_simple2-both.pkl')
save_clean_data(train, 'english-japanese_simple2-train.pkl')
save_clean_data(test, 'english-japanese_simple2-test.pkl')

Saved: english-japanese_simple2-both.pkl
Saved: english-japanese_simple2-train.pkl
Saved: english-japanese_simple2-test.pkl


In [8]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint

import pydot

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

# one hot encode target sequence
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
	model = Sequential()
	model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
	model.add(LSTM(n_units))
	model.add(RepeatVector(tar_timesteps))
	model.add(LSTM(n_units, return_sequences=True))
	model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
	return model

# load datasets
dataset = load_clean_sentences('english-japanese_simple2-both.pkl')
train = load_clean_sentences('english-japanese_simple2-train.pkl')
test = load_clean_sentences('english-japanese_simple2-test.pkl')

# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
print('Japanese Vocabulary Size: %d' % ger_vocab_size)
print('Japanese Max Length: %d' % (ger_length))

# prepare training data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = encode_output(trainY, eng_vocab_size)
# prepare validation data
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])
testY = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
testY = encode_output(testY, eng_vocab_size)

# define model
model = define_model(ger_vocab_size, eng_vocab_size, ger_length, eng_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
# summarize defined model
print(model.summary())
plot_model(model, to_file='japmodel_simple2.png', show_shapes=True)
# fit model
filename = 'japmodel_simple2.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
model.fit(trainX, trainY, epochs=25, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)


English Vocabulary Size: 2716
English Max Length: 7
Japanese Vocabulary Size: 4740
Japanese Max Length: 19
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 19, 256)           1213440   
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 7, 256)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 7, 256)            525312    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 7, 2716)           698012    
Total params: 2,962,076
Trainable params: 2,962,076
Non-trainable params: 0
_________________________________________________________________
None
Train on 9000 sample

In [1]:
from pickle import load
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate target given source sequence
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

# evaluate the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
	actual, predicted = list(), list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, eng_tokenizer, source)
		raw_target, raw_src = raw_dataset[i]
		if i < 10:
			print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
		actual.append([raw_target.split()])
		predicted.append(translation.split())

# load datasets
dataset = load_clean_sentences('english-japanese_simple2-both.pkl')
train = load_clean_sentences('english-japanese_simple2-train.pkl')
test = load_clean_sentences('english-japanese_simple2-test.pkl')
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
# prepare data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])

# load model
model = load_model('japmodel.h5')
# test on some training sequences
print('train')
evaluate_model(model, eng_tokenizer, trainX, train)
# test on some test sequences
print('test')
evaluate_model(model, eng_tokenizer, testX, test)
# print(train)

C:\Users\NitishTalekar\Applictions\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


train
src=[と にかく やれ よ], target=[just do that], predicted=[to my my]
src=[顔色 が わるい です よ], target=[you look pale], predicted=[i dont the the]
src=[英語 を 勉強 し ましょ う], target=[lets study english], predicted=[in she she she she]
src=[誰], target=[who is it], predicted=[hobby]
src=[この 本 は 彼女 の です], target=[this book is hers], predicted=[a do hows]
src=[おなか が 痛い], target=[my stomach hurts], predicted=[up me me]
src=[やめろよ], target=[stop it], predicted=[got tom]
src=[トム の ため に 席 を とっ て おい た], target=[i saved tom a seat], predicted=[i are i i i]
src=[ちょっと 目 を 閉じ て], target=[just close your eyes], predicted=[ready it]
src=[戦争 は よく ない], target=[war isnt good], predicted=[to here good]
test
src=[私 は 学生 です], target=[i am a student], predicted=[to read lost lost]
src=[彼 は ウェールズ 出身 です], target=[he comes from wales], predicted=[you is is yourself]
src=[６ 時半 に 戻り ます], target=[ill be back at], predicted=[she a just im has]
src=[これ を 読ん で], target=[read this], predicted=[she a no im]
src=[彼 に 電話 し た], targe

In [11]:
testmodel = load_model('japmodel_simple.h5')
print("Model Loaded")

Model Loaded


In [5]:
from pickle import load
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
import pandas as pd
import numpy as np

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

dataset = load_clean_sentences('english-japanese_simple2-both.pkl')
train = load_clean_sentences('english-japanese_simple2-train.pkl')
test = load_clean_sentences('english-japanese_simple2-test.pkl')
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
# prepare data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])

In [6]:
model = load_model('japmodel_simple.h5')
preds = model.predict_classes(testX.reshape((testX.shape[0],testX.shape[1])))

def get_word(n, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == n:
            return word
    return None

preds_text = []
for i in preds:
    temp = []
    for j in range(len(i)):
        t = get_word(i[j], eng_tokenizer)
        if j > 0:
            if (t == get_word(i[j-1], eng_tokenizer)) or (t == None):
                temp.append('')
            else:
                temp.append(t)
        else:
            if(t == None):
                temp.append('')
            else:
                temp.append(t) 

    preds_text.append(' '.join(temp))
    
pred_df = pd.DataFrame({'jap':test[:,1],'actual' : test[:,0], 'predicted' : preds_text})
pred_df.sample(15)

,jap,actual,predicted
665,トム は 決し て 笑わ ない,tom never laughs,i dont tom still
24,これ 誰 に もらっ た ん です か,who gave you this,i like goes
556,彼女 は と て も 賢い人 だ,she is very wise,i do good
48,おしゃれ を し ない といけ ない,i have to dress up,to made his
435,この ネクタイ は いくら です か,how much is this tie,every tom it
30,彼女 は 遅刻 魔 だ,she tends to be late,with tom sleepy
939,それ は 黒く あり ませ んか,is it not black,who it
19,とって も 好き よ,i like it a lot,i what im ill
462,トム は 反社会 的 人間 だ,tom is a sociopath,it is tom some
622,値段 が 高すぎ です,it is too expensive,to
